### a generic physics simulation engine

Step 1: make simple simulation working.

components: 
- visual rendering page with different speed controls 
- handling two kinds of points (fixed-position ie force projecting only and force projecting & force subjected)
- application of forces on different sets of points

step 1.1: turn point_sets into class

Step 2: vectorize operations
Step 3: format as database

In [40]:
import pymunk

In [19]:
# simple simulation stub, unoptimized/etc
point_sets = [\
    { 'force_receiving':True,
      'force_projecting':True,
      'data': [[0,1], [2,3]]},
    { 'force_receiving':False,
      'force_projecting':True,
      'data': [[1,1], [-2,1]]},        
    ] # later: make as a class that can be exported as json
restructured_point_sets = []
for set_ in point_sets:
    for data_point in set_['data']:
        restructured_point_sets += [\
                (data_point, set_['force_receiving'], 
                 set_['force_projecting'])]

In [38]:
def same_point(a,b): return a==b
def sub(b,a): return [b[0][0]-a[0][0], b[0][1]-a[0][1]] # later: numpy vectorize
def eucl(x): return (x[0]**2+x[1]**2)**0.5
def normz_vec(vec, dist_fn):
    epsilon = 1e-6
    dist = dist_fn(vec)
    if abs(dist-1)<epsilon: 
        return vec
    return [vec[0]/dist, vec[1]/dist]
def Round(x,N=4, base=10): # quick n dirty
    if isinstance(x,int): return x
    if isinstance(x, list):
        return [((xi*base**N)//1)/base**N for xi in x]
    if isinstance(x,tuple): # hacky
        return tuple([((xi*base**N)//1)/base**N for xi in x])
    return ((x*base**N)//1)/base**N
def div(x,s): return [xi/s for xi in x]
def accum(a,b): return [a[i]+b[i] for i in range(len(a))]
def apply_force(vec, factor):
    if factor=='inverse_distance_squared':
        magn= eucl(vec)
        unit_dir_vec = normz_vec(diff_vec, eucl)
        return div(unit_dir_vec, magn**2)
    raise Exception(NotImplemented)
    
for point in restructured_point_sets:
    if point[1]==False: continue # not force receiving
    net_force=[0,0]
    points_forces = []
    for other_point in restructured_point_sets:
        if same_point(point, other_point): 
            #print(point[0], other_point[0], '**')
            continue
        else:
            #print(point[0], other_point[0] )
            pass
        diff_vec = sub(other_point, point)
        magn = eucl(diff_vec)
        unit_dir_vec = normz_vec(diff_vec, eucl)
        print(point[0], other_point[0],'\t', diff_vec, '\t', Round(magn,2),\
              '\t', Round(unit_dir_vec))
        
        force_vec = apply_force(diff_vec, 'inverse_distance_squared')
        points_forces += force_vec
        net_force = accum(net_force, force_vec)
    print(net_force)
        
      

[0, 1] [2, 3] 	 [2, 2] 	 2.82 	 [0.7071, 0.7071]
[0, 1] [1, 1] 	 [1, 0] 	 1.0 	 [1.0, 0.0]
[0, 1] [-2, 1] 	 [-2, 0] 	 2.0 	 [-1.0, 0.0]
[0.8383883476483185, 0.08838834764831842]
[2, 3] [0, 1] 	 [-2, -2] 	 2.82 	 [-0.7072, -0.7072]
[2, 3] [1, 1] 	 [-1, -2] 	 2.23 	 [-0.4473, -0.8945]
[2, 3] [-2, 1] 	 [-4, -2] 	 4.47 	 [-0.8945, -0.4473]
[-0.2225524262983058, -0.2896344656232995]


import pymunk               # Import pymunk..

space = pymunk.Space()      # Create a Space which contain the simulation
space.gravity = 0,-981      # Set its gravity

body = pymunk.Body()        # Create a Body
body.position = 50,100      # Set the position of the body

poly = pymunk.Poly.create_box(body) # Create a box shape and attach to body
poly.mass = 10              # Set the mass on the shape
space.add(body, poly)       # Add both body and shape to the simulation

print_options = pymunk.SpaceDebugDrawOptions() # For easy printing

for _ in range(10):               # Infinite loop simulation
    space.step(0.02)        # Step the simulation one step forward
    space.debug_draw(print_options) # Print the state of the simulation